In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import cm


In [5]:

def conditional_bivariate_gaussian(mean, covariance, y1_observed, filename_prefix):
    """
    Produce two figures:
    1) Bivariate Gaussian pdf (contour plot) with a vertical line at y1_observed.
    2) Univariate Gaussian pdf showing p(y2|y1_observed).

    Parameters
    ----------
    mean : array-like of shape (2,)
        Mean vector [mu1, mu2].
    covariance : array-like of shape (2, 2)
        Covariance matrix [[Sigma11, Sigma12], [Sigma21, Sigma22]].
    y1_observed : float
        Observed value of y1, at which we condition.
    filename_prefix : str
        Prefix for the output filenames.
    """
    # Unpack parameters for convenience
    mu1, mu2 = mean
    Sigma11 = covariance[0, 0]
    Sigma12 = covariance[0, 1]
    Sigma21 = covariance[1, 0]
    Sigma22 = covariance[1, 1]

    # -------------------------------------------------------------------------
    # 1) Bivariate Gaussian and vertical line at y1 = y1_observed
    # -------------------------------------------------------------------------
    # Create a grid of points for contour plotting
    x = np.linspace(mu1 - 4, mu1 + 4, 300)
    y = np.linspace(mu2 - 4, mu2 + 4, 300)
    X, Y = np.meshgrid(x, y)

    # Compute the pdf values on the grid
    inv_cov = np.linalg.inv(covariance)
    det_cov = np.linalg.det(covariance)

    # Vectorize computation for speed
    xy = np.dstack([X - mu1, Y - mu2])
    Z = np.empty(X.shape)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            z_vec = xy[i, j]
            Z[i, j] = np.exp(-0.5 * z_vec @ inv_cov @ z_vec)

    # Normalizing constant for bivariate Gaussian
    norm_const = 1.0 / (2.0 * np.pi * np.sqrt(det_cov))
    Z = Z * norm_const

    # Plot the contours
    #plt.figure(figsize=(4, 4))
    contour_levels = 10
    plt.contour(X, Y, Z, levels=contour_levels, cmap=cm.viridis)
    plt.axvline(x=y1_observed, color='red', linestyle='--', label=r'$y_1 = %.2f$' % y1_observed)

    plt.xlabel(r'$y_1$')
    plt.ylabel(r'$y_2$')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"fig/cond_{filename_prefix}_bivariate.pdf")
    plt.close()

    # -------------------------------------------------------------------------
    # 2) Univariate Gaussian for y2 given y1_observed
    # -------------------------------------------------------------------------
    # The conditional distribution of Y2 given Y1 = y1_observed is:
    #
    #   Y2 | Y1 = y1_observed ~ Normal(mean_cond, var_cond)
    #
    # where:
    #   mean_cond = mu2 + (Sigma21 / Sigma11) * (y1_observed - mu1)
    #   var_cond  = Sigma22 - (Sigma21 * Sigma12) / Sigma11

    mean_cond = mu2 + (Sigma21 / Sigma11) * (y1_observed - mu1)
    var_cond = Sigma22 - (Sigma21 * Sigma12) / Sigma11
    std_cond = np.sqrt(var_cond)

    # Create grid for y2
    y2 = np.linspace(mean_cond - 4 * std_cond, mean_cond + 4 * std_cond, 300)

    # Compute pdf of the conditional normal
    pdf_y2_cond = (
        1.0 / (np.sqrt(2.0 * np.pi * var_cond))
        * np.exp(-0.5 * ((y2 - mean_cond) ** 2) / var_cond)
    )

    # Plot the conditional pdf
    #plt.figure(figsize=(4, 4))
    plt.plot(y2, pdf_y2_cond, color='blue')
    plt.xlabel(r'$y_2$')
    plt.ylabel(r'pdf $(y_2 \mid y_1=%.2f)$' % y1_observed)
    # force the x-axis to be between the values -2.5 and 2.5
    plt.xlim(-2.5, 2.5)

    plt.tight_layout()
    plt.savefig(f"fig/cond_{filename_prefix}_conditional.pdf")
    plt.close()


In [3]:
mean = np.array([0, 0])
covariance = np.array([[1, 0.9], [0.9, 1]])

conditional_bivariate_gaussian(mean, covariance, -1.0, "ex1")
conditional_bivariate_gaussian(mean, covariance, 0.0, "ex2")
conditional_bivariate_gaussian(mean, covariance, 1.0, "ex3")